# RAG application using open-source models
Code adapted from https://github.com/svpino/llm

In [1]:
!pip install pypdf
!pip install langchain_community
!pip install langchain[docarray]
!pip  install pydantic==1.10.9
!pip install langchain




You should consider upgrading via the 'd:\unistra\unistra\recherche\ragbot\env\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\unistra\unistra\recherche\ragbot\env\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\unistra\unistra\recherche\ragbot\env\scripts\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1


You should consider upgrading via the 'd:\unistra\unistra\recherche\ragbot\env\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\unistra\unistra\recherche\ragbot\env\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "llama3:8b"
MODEL = "llama3:8b-instruct-q4_0"


In [3]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

#from langchain_openai.embeddings import OpenAIEmbeddings
#from langchain_openai.chat_models import ChatOpenAI

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

#model.invoke("Tell me a joke")

In [5]:
#model.invoke("Raconte-moi une blague")

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
#chain.invoke("Tell me a joke")

In [5]:
from langchain.prompts import PromptTemplate

template = """
You are an expert in ecology and environmental issues that speaks English and French. You will answer in French to questions from journalists during a press conference. Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nYou are an expert in ecology and environmental issues that speaks English and French. You will answer in French to questions from journalists during a press conference. Answer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [6]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'Bonjour ! Mon nom est Santiago.'

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ERP_FR.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='DOCUMENT DE \nRÉFÉRENCE POUR \nLA RESTAURATION \nDES ÉCOSYSTÈMES   \n \nUN GUIDE PRATIQUE POUR GUÉRIR  \nLA PLANÈTE  \ndecadeonrestoration.org Développé pour La Journée mondiale de l’environnement 2021\nPour lancer la Décennie des Nations Unies pour la Restauration des Écosystèmes (2021-2030)', metadata={'source': 'ERP_FR.pdf', 'page': 0}),
 Document(page_content='2Introduction ................................................................................................. 3\nPourquoi les écosystèmes sont-ils si importants .......................................... 5\nComment rejoindre la #GénérationRestauration ............................................ 6\nInformez-vous! ............................................................................................ 7\nAction de restauration ................................................................................ 10\n Principes et recommandations ..........................................................

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke("restauration d'écosystèmes")

[Document(page_content='11Nous allons à présent plonger dans les profondeurs de \nsept écosystèmes. Mais ne soyez pas intimidé, \nchacun peut mener des actions en fonction de chaque \nécosystème. Informez-vous sur ceux qui vous tiennent \nà cœur et découvrez ce que vous pouvez faire.ÉCOSYSTÈME PAR ÉCOSYSTÈME: LES DIFFÉRENTS VISAGES DE LA \nRESTAURATION\nFORÊTS ET ARBRES RIVIÈRES ET LACS VILLES MONTAGNES TOURBIÈRES OCÉANS ET ZONES CÔTIÈRES TERRES AGRICOLES ET PRAIRIES', metadata={'source': 'ERP_FR.pdf', 'page': 10}),
 Document(page_content='sur vos T-shirts ou affiches. Pour en savoir plus, rendez-\nvous sur notre guide de la marque.\nBien que nous voulions tous célébrer la Journée \nmondiale de l’environnement et la Décennie des Nations \nUnies, respectez toutes les restrictions relatives à la \nCOVD-19 en vigueur où vous vivez et organisez des \ndiscussions et des événements en ligne pour que \nchacun soit protégé.\nVoici certaines des considérations à avoir à l’esprit \nlorsque vous 

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "Qu'est-ce que la restauration d'écosystèmes ?",
    "Quels sont les avantages des écosystèmes ?",
    "Comment les entreprises peuvent aider aux écosystèmes ?",
    "Qui a publié les principes de développement et de gestion des projets ?",
    "What is the goal of Terramatch?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Qu'est-ce que la restauration d'écosystèmes ?
Answer: Bonjour ! Selon les informations fournies, la restauration d'écosystèmes concerne l'effort collectif pour réduire les pressions exercées sur les océans et les zones côtières, afin qu'ils puissent se rétablir naturellement et avec notre aide. Il s'agit de restaurer la végétation au-dessus et en dessous de l'eau, protéger et restaurer les écosystèmes côtiers tels que les marais salés, les mangroves, les récifs coralliens, les herbiers marins et les bancs de crustacés.

Cela implique également de réduire les déchets plastiques, de promouvoir une agriculture résiliente et d'utiliser des techniques durables pour restaurer les sols et la biodiversité. Enfin, il est important de rendre ces écosystèmes et ces communautés plus résilientes face aux changements mondiaux.

En somme, la restauration d'écosystèmes consiste à réduire les pressions exercées sur les océans et les zones côtières pour permettre leur rétablissement naturel et

In [ ]:
chain.batch([{"question": q} for q in questions])

In [13]:
for s in chain.stream({"question": "Qu'est-ce que ça veut dire PNUE ?"}):
    print(s, end="", flush=True)

La réponse à votre question ! 😊

PNUE signifie " Programme des Nations Unies pour l'environnement". Il s'agit d'un programme de l'Organisation des Nations Unies (ONU) qui vise à promouvoir la conservation et la mise en valeur de l'environnement.

Dans ce contexte, le document ERP_FR.pdf semble être un guide sur la restauration écologique, notamment pour les écosystèmes de montagnes et les zones côtières. Le programme PNUE est donc peut-être une organisation qui a créé ce guide pour aider à restaurer l'environnement et à préserver la biodiversité.

Si vous avez des questions spécifiques sur le contenu du document ou sur la restauration écologique en général, n'hésitez pas à me demander ! 😊

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents